In [ ]:
# ==============================================================
# 0. Install dependencies
# ==============================================================

#!pip install --quiet --upgrade openai sentence-transformers faiss-cpu tiktoken

# ==============================================================
# 1. Imports and configuration
# ==============================================================

import os
import time
import openai
import faiss
import numpy as np
from google.colab import userdata
from sentence_transformers import SentenceTransformer

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()


# ==============================================================
# 2. Helper: LLM call wrapper (compatible with modern API)
# ==============================================================

def call_llm(system_prompt: str,
             user_prompt: str,
             model: str = "gpt-4o-mini",
             temperature: float = 0.2,
             max_tokens: int = 800):
    """Unified OpenAI LLM call for chat completion."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]
    for attempt in range(3):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            print(f"LLM call failed (attempt {attempt+1}): {e}")
            time.sleep(2)
    raise RuntimeError("LLM calls failed after retries")

# ==============================================================
# 3. Vector Memory Implementation
# ==============================================================

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

class VectorMemory:
    def __init__(self, dim=384):
        self.dim = dim
        self.index = faiss.IndexFlatL2(dim)
        self.metadata = []

    def add(self, texts, metadatas=None):
        if isinstance(texts, str):
            texts = [texts]
        vecs = embed_model.encode(texts, convert_to_numpy=True)
        self.index.add(vecs)
        if metadatas is None:
            metadatas = [{} for _ in texts]
        self.metadata.extend(metadatas)

    def query(self, query_text, k=3):
        vec = embed_model.encode([query_text], convert_to_numpy=True)
        D, I = self.index.search(vec, k)
        results = []
        for idx in I[0]:
            if idx < len(self.metadata):
                results.append(self.metadata[idx].get('text', ''))
        return results

memory = VectorMemory(dim=embed_model.get_sentence_embedding_dimension())
memory.add([
    "Anil Pathak's Learning academy - Q3 revenue increased by 12% compared to Q2.",
    "Primary contact for Training Project Agentic AI is Anil Pathak (anil.pathak@ieee.org).",
    "Agentic AI Training launch event scheduled on November 8, 2025.",
])

# ==============================================================
# 4. Define Agent Roles
# ==============================================================

class PlannerAgent:
    def __init__(self):
        self.system = "You are PlannerAgent. Break a goal into concise numbered subtasks."
    def plan(self, user_goal: str):
        prompt = f"User goal: {user_goal}\n\nReturn a short numbered list (1., 2., 3., ...) of subtasks."
        resp = call_llm(self.system, prompt)
        subtasks = [line.strip() for line in resp.split('\n') if line.strip()]
        return subtasks

class ResearchAgent:
    def __init__(self, memory: VectorMemory):
        self.system = "You are ResearchAgent. Use memory facts to answer the query."
        self.memory = memory
    def research(self, query: str):
        hits = self.memory.query(query, k=3)
        if not hits:
            return "(No memory found — would call external data source in production.)"
        content = '\n'.join([f"- {h}" for h in hits])
        prompt = f"Context:\n{content}\n\nQuestion: {query}\nProvide a short factual summary."
        return call_llm(self.system, prompt)

class WriterAgent:
    def __init__(self):
        self.system = "You are WriterAgent. Compose professional summaries or reports."
    def write(self, outline: str, research_notes: str):
        prompt = f"Outline:\n{outline}\n\nResearch Notes:\n{research_notes}\n\nCreate a concise executive summary."
        return call_llm(self.system, prompt)

# Instantiate agents
planner = PlannerAgent()
researcher = ResearchAgent(memory)
writer = WriterAgent()

# ==============================================================
# 5. Orchestrator: pipeline controller
# ==============================================================

class Orchestrator:
    def __init__(self, planner, researcher, writer):
        self.planner = planner
        self.researcher = researcher
        self.writer = writer

    def run(self, user_goal: str):
        plan = self.planner.plan(user_goal)
        print("Planner produced subtasks:\n", "\n".join(plan))

        all_research = []
        for i, subtask in enumerate(plan, 1):
            query = f"Information needed to {subtask}"
            research = self.researcher.research(query)
            print(f"\nResearch for subtask {i}:\n{research}")
            all_research.append((subtask, research))

        outline = "\n".join([f"{i}. {s}" for i, s in enumerate(plan, 1)])
        research_notes = "\n\n".join([f"Subtask: {s}\nNotes: {r}" for s, r in all_research])
        final = self.writer.write(outline, research_notes)
        return final

orchestrator = Orchestrator(planner, researcher, writer)

# ==============================================================
# 6. Run the end-to-end demo
# ==============================================================

user_goal = (
    "Create a 3-slide executive summary of Anil Pathak's Learning academy Q3 performance, "
    "list 5 action items for the training launch, and draft an email to stakeholders."
)

print("Running orchestrator...\n")
result = orchestrator.run(user_goal)

print("\n--- FINAL OUTPUT ---\n")
print(result)


Running orchestrator...

Planner produced subtasks:
 1. Gather data on Anil Pathak's Learning Academy Q3 performance metrics (e.g., enrollment numbers, course completion rates, feedback scores).
2. Create a concise 3-slide executive summary highlighting key performance indicators, successes, and areas for improvement.
3. Identify and outline 5 actionable items for the upcoming training launch (e.g., marketing strategies, resource allocation, scheduling).
4. Draft an email to stakeholders summarizing the Q3 performance, the action items for the training launch, and any relevant updates or requests for feedback.
5. Review and finalize the executive summary slides and email for clarity and professionalism before distribution.

Research for subtask 1:
To gather data on Anil Pathak's Learning Academy Q3 performance metrics, you would need the following information:

1. **Enrollment Numbers**: Total number of students enrolled in courses during Q3.
2. **Course Completion Rates**: Percentage 